Top 15 Board Comment Distribution&Comments Validity Distribution

In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import os
from matplotlib.font_manager import FontProperties
import traceback


# 1. 标题预处理+看板名提取
def preprocess_title(title):
    """预处理标题：去除前缀+归一化符号"""
    if pd.isna(title):
        return ""
    title = str(title).strip()
    # 去除Re/Fw前缀
    title = re.sub(r'^[Rr][Ee][:\s]*', '', title)
    title = re.sub(r'^[Ff][Ww][:\s]*', '', title)
    # 归一化符号（全角/半角连接符、冒号转为空格）
    title = re.sub(r'[:：\-_\—\s]+', ' ', title)
    return title

def extract_board_name(title_series):
    """提取看板名：兼容括号格式+文本格式（閱讀文章 - 看板 XXX）"""
    def extract_single(title):
        title = preprocess_title(title)
        if not title:
            return None
        
        # 规则1：括号格式（优先级最高）
        bracket_pattern = r'^[\[\【\［\{｛](.+?)[\]\】\］\}\}]'
        bracket_match = re.search(bracket_pattern, title)
        if bracket_match:
            board = bracket_match.group(1).strip()
            if board and not board.isdigit() and 1 <= len(board) <= 30:
                return board
        
        # 规则2：完整文本格式（閱讀文章 - 看板 XXX）
        full_text_pattern = r'閱讀文章\s+看板\s+([^\s]+)'
        full_match = re.search(full_text_pattern, title)
        if full_match:
            board = full_match.group(1).strip()
            if board and not board.isdigit() and 1 <= len(board) <= 30:
                return board
        
        # 规则3：简化文本格式（看板 XXX）
        simple_text_pattern = r'看板\s+([^\s]+)'
        simple_match = re.search(simple_text_pattern, title)
        if simple_match:
            board = simple_match.group(1).strip()
            if board and not board.isdigit() and 1 <= len(board) <= 30:
                return board
        
        return None
    
    board_names = title_series.apply(extract_single)
    return board_names


# 2. 字体配置+低频看板名过滤

def setup_traditional_chinese_font():
    """配置繁体中文字体"""
    plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'PingFang SC']
    plt.rcParams['axes.unicode_minus'] = False
    return FontProperties(family='Arial Unicode MS')

def filter_low_frequency_boards(board_comment_count, min_count=2):
    """过滤低频看板名（减少噪声）"""
    filtered = board_comment_count[board_comment_count['comment_count'] >= min_count].copy()
    removed = len(board_comment_count) - len(filtered)
    print(f"过滤低频看板名（<{min_count}条评论）：{removed} 个")
    return filtered


# 3. 数据加载与处理
def load_and_process_data(comment_path, post_path):
    """加载评论表和帖子表，处理数据并统计看板评论数"""
    # 加载评论表
    print(f"1. 加载评论表：{comment_path}")
    try:
        df_comment = pd.read_csv(comment_path)
    except Exception as e:
        raise FileNotFoundError(f"评论表加载失败：{str(e)}")
    total_comment_raw = len(df_comment)
    print(f"   原始评论数：{total_comment_raw:,} 条")
    
    # 加载帖子表
    print(f"\n2. 加载帖子表：{post_path}")
    try:
        df_post = pd.read_csv(post_path)
    except Exception as e:
        raise FileNotFoundError(f"帖子表加载失败：{str(e)}")
    total_post_raw = len(df_post)
    print(f"   原始帖子数：{total_post_raw:,} 条")

    # 清洗帖子表（过滤空标题、短标题）
    print("\n3. 清洗帖子表数据")
    df_post = df_post[
        df_post['title'].notna() & 
        (df_post['title'].astype(str).str.strip().str.len() >= 0)
    ]
    # 去重Post ID
    post_dup_count = df_post['post_id'].duplicated().sum()
    df_post = df_post.drop_duplicates(subset=['post_id'], keep='first')
    post_clean_count = len(df_post)
    print(f"   清洗后帖子数：{post_clean_count:,} 条（删除{total_post_raw - post_clean_count:,}条）")
    print(f"   去重Post ID数：{post_dup_count:,} 个")
    
    # 提取看板名
    print("\n4. 提取帖子看板名")
    df_post['board'] = extract_board_name(df_post['title'])
    board_valid_count = df_post['board'].notna().sum()
    board_invalid_count = df_post['board'].isna().sum()
    print(f"   有效看板名数：{board_valid_count:,} 条（{board_valid_count/post_clean_count*100:.1f}%）")
    print(f"   无效看板名数：{board_invalid_count:,} 条（{board_invalid_count/post_clean_count*100:.1f}%）")
    
    # 关联评论表和帖子表（按Post ID）
    print("\n5. 关联评论表与帖子表")
    df_comment['post_id_str'] = df_comment['post_id'].astype(str).fillna('')
    df_post['post_id_str'] = df_post['post_id'].astype(str).fillna('')
    df_merged = pd.merge(
        df_comment,
        df_post[['post_id', 'board', 'post_id_str']],
        on='post_id',
        how='inner'
    )
    merged_count = len(df_merged)
    print(f"   关联后评论数：{merged_count:,} 条")
    
    # 清洗关联后数据（过滤无效看板名、空评论内容）
    print("\n6. 清洗关联后数据")
    df_merged = df_merged[df_merged['board'].notna()]
    df_merged['content_str'] = df_merged['content'].astype(str).fillna('').str.strip()
    df_merged = df_merged[df_merged['content_str'] != '']
    clean_count = len(df_merged)
    print(f"   清洗后有效评论数：{clean_count:,} 条")

    
    # 统计看板评论数
    print("\n7. 统计各看板评论数")
    board_comment_count = df_merged.groupby('board').size().reset_index(name='comment_count')
    board_comment_count = board_comment_count.sort_values('comment_count', ascending=False).reset_index(drop=True)
    # 过滤低频看板名
    board_comment_count = filter_low_frequency_boards(board_comment_count)
    print(f"   有效看板数量：{len(board_comment_count)} 个")
    print(f"   Top 5 看板：\n{board_comment_count.head(5).to_string(index=False)}")
    
    return board_comment_count, clean_count, total_comment_raw



# 4. 可视化函数：绘制看板评论数柱状图

def plot_board_comment_chart(board_comment_count, total_valid_comments, tc_font):
    """绘制Top 15看板评论数柱状图"""
    print("\n8. 生成可视化图表")
    top_n = 15
    plot_data = board_comment_count.head(top_n) if len(board_comment_count) > top_n else board_comment_count
    
    # 绘制图表
    plt.figure(figsize=(14, 8))
    bars = plt.bar(
        plot_data['board'],
        plot_data['comment_count'],
        color='#3498DB',
        alpha=0.8,
        edgecolor='white'
    )
    # 设置标题和标签
    plt.title(
        f'PTT Top 15 Board Comment Distribution (Number of Valid Comments:{total_valid_comments:,} posts)',
        fontsize=16,
        fontweight='bold',
        fontfamily='DejaVu Sans'
    )
    plt.xlabel('Board Name', fontsize=14, fontproperties=tc_font)
    plt.ylabel('Number of Comment', fontsize=14, fontproperties=tc_font)
    plt.xticks(rotation=45, ha='right', fontproperties=tc_font)
    # 添加数值标签
    for bar in bars:
        height = bar.get_height()
        plt.text(
            bar.get_x() + bar.get_width()/2,
            height + height*0.01,
            f'{int(height):,}',
            ha='center',
            va='bottom',
            fontproperties=tc_font,
            fontsize=10
        )
    # 网格与布局
    plt.grid(axis='y', alpha=0.3, linestyle='--')
    plt.tight_layout()
    # 保存图表
    save_path = os.path.join(os.path.expanduser('~'), '/Users/apple/Desktop/data/visual_new/ptt_board_comment(1).png')
    plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='white')
    plt.close()
    print(f"   图表已保存至：{save_path}")
    
    return board_comment_count

# 绘制有效/无效评论分布饼图
def plot_comment_validity_pie(total_comment_raw, total_comments, tc_font):

    print("\n11. 生成评论有效性分布饼图")
    # 数据准备
    valid_count = total_comments
    invalid_count = total_comment_raw - valid_count  # 总无效数（含未匹配+清洗删除）
    labels = ['Valid Comment', 'Invalid Comment']
    sizes = [valid_count, invalid_count]
    colors = ['#2ECC71', '#E74C3C']
    explode = (0.05, 0)  # 突出有效评论
    # 绘制饼图
    plt.figure(figsize=(10, 8))
    plt.pie(
        sizes,
        explode=explode,
        labels=labels,
        colors=colors,
        autopct='%1.1f%%',
        shadow=True,
        startangle=90,
        textprops={'fontsize': 12, 'fontproperties': setup_traditional_chinese_font()}
    )
    # 设置标题
    plt.title(
        f'PTT Comments Validity Distribution (Total:{total_comment_raw:,} posts)',
        fontsize=16,
        fontweight='bold',
        fontfamily='DejaVu Sans'
    )
    plt.axis('equal')  # 保持饼图为正圆形
    # 保存图表
    save_path = os.path.join(os.path.expanduser('~'), '/Users/apple/Desktop/data/visual_new/ptt_comment_validity(1).png')
    plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='white')
    plt.close()
    print(f"   评论有效性分布图表已保存至：{save_path}")



# 5. 执行流程
def main():
    try:
        print("=== 开始处理PTT评论数据 ===")
        print("="*50)
        # 配置文件路径
        COMMENT_PATH = "/Users/apple/Desktop/data/data_clean/ptt_comments_clean(1).csv"
        POST_PATH = "/Users/apple/Desktop/data/data_clean/ptt_posts_clean(1).csv"
        
        # 检查文件是否存在
        assert os.path.exists(COMMENT_PATH), f"评论表文件不存在：{COMMENT_PATH}"
        assert os.path.exists(POST_PATH), f"帖子表文件不存在：{POST_PATH}"
        
        # 配置字体
        tc_font = setup_traditional_chinese_font()
        
        # 加载处理数据并统计看板评论数
        board_comment_count, total_valid, total_raw = load_and_process_data(COMMENT_PATH, POST_PATH)
        
        # 绘制可视化图表
        plot_board_comment_chart(board_comment_count, total_valid, tc_font)
        plot_comment_validity_pie(total_raw, total_valid, tc_font)
        
        print("\n" + "="*50)
        print("=== 数据处理完成 ===")
        print(f"总原始评论数：{total_raw:,} 条")
        print(f"总有效评论数：{total_valid:,} 条")
        print(f"有效看板数量：{len(board_comment_count)} 个")
        
    except AssertionError as e:
        print(f"\n断言错误：{str(e)}")
    except FileNotFoundError as e:
        print(f"\n文件错误：{str(e)}")
    except Exception as e:
        print(f"\n执行错误：")
        print(f"   类型：{type(e).__name__}")
        print(f"   信息：{str(e)[:200]}")
        traceback.print_exc(limit=5)
    finally:
        print("\n=== 程序结束 ===")

if __name__ == "__main__":
    main()

=== 开始处理PTT评论数据 ===
1. 加载评论表：/Users/apple/Desktop/data/data_clean/ptt_comments_clean(1).csv
   原始评论数：640,237 条

2. 加载帖子表：/Users/apple/Desktop/data/data_clean/ptt_posts_clean(1).csv
   原始帖子数：25,739 条

3. 清洗帖子表数据
   清洗后帖子数：25,739 条（删除0条）
   去重Post ID数：0 个

4. 提取帖子看板名
   有效看板名数：24,498 条（95.2%）
   无效看板名数：1,241 条（4.8%）

5. 关联评论表与帖子表
   关联后评论数：637,815 条

6. 清洗关联后数据
   清洗后有效评论数：626,671 条

7. 统计各看板评论数
过滤低频看板名（<2条评论）：30 个
   有效看板数量：578 个
   Top 5 看板：
board  comment_count
   新聞         137161
   問卦          95567
   閒聊          62216
   問題          47722
   爆卦          30432

8. 生成可视化图表
   图表已保存至：/Users/apple/Desktop/data/visual_new/ptt_board_comment(1).png

11. 生成评论有效性分布饼图
   评论有效性分布图表已保存至：/Users/apple/Desktop/data/visual_new/ptt_comment_validity(1).png

=== 数据处理完成 ===
总原始评论数：640,237 条
总有效评论数：626,671 条
有效看板数量：578 个

=== 程序结束 ===
